# Questions

In [ ]:
#Produce a timestamp for 1 a.m. on the 31st of August 2012
SELECT TIMESTAMP '2012-08-31 01:00:00'

#Find the result of subtracting the timestamp '2012-07-30 01:00:00' from the timestamp '2012-08-31 01:00:00'
SELECT TIMESTAMP '2012-08-31 01:00:00' - TIMESTAMP '2012-07-30 01:00:00' AS interval

#Produce a list of all the dates in October 2012. They can be output as a timestamp (with time set to midnight) or a date.
SELECT GENERATE_SERIES(TIMESTAMP '2012-10-01', TIMESTAMP '2012-10-31', INTERVAL '1 day') AS timestamp

#Get the day of the month from the timestamp '2012-08-31' as an integer.
SELECT EXTRACT(DAY FROM TIMESTAMP '2012-08-31') AS date_part

#Work out the number of seconds between the timestamps '2012-08-31 01:00:00' and '2012-09-02 00:00:00'
SELECT EXTRACT(EPOCH FROM (TIMESTAMP '2012-09-02 00:00:00' - '2012-08-31 01:00:00')) AS seconds

#For each month of the year in 2012, output the number of days in that month. Format the output 
#as an integer column containing the month of the year, and a second column containing an interval data type.
SELECT EXTRACT(month FROM cal.month) AS month,
	(cal.month + interval '1 month') - cal.month AS length
FROM (SELECT GENERATE_SERIES(TIMESTAMP '2012-01-01', TIMESTAMP '2012-12-01', INTERVAL '1 month') AS month) cal
ORDER BY month

#For any given timestamp, work out the number of days remaining in the month. The current day should count as a 
#whole day, regardless of the time. Use '2012-02-11 01:00:00' as an example timestamp for the purposes of making
# the answer. Format the output as a single interval value.
SELECT (DATE_TRUNC('month', ts.testts) + INTERVAL '1 month') -
	DATE_TRUNC('day', ts.testts) AS remaining
FROM (SELECT TIMESTAMP '2012-02-11 01:00:00' AS testts) ts

#Return a list of the start and end time of the last 10 bookings (ordered by the time at which they end, 
#followed by the time at which they start) in the system.
SELECT starttime, starttime + slots*(interval '30 minutes') endtime
FROM cd.bookings
ORDER BY endtime DESC, starttime DESC
LIMIT 10

#Return a count of bookings for each month, sorted by month
SELECT DATE_TRUNC('month', starttime) AS month, count(*)
FROM cd.bookings
GROUP BY month
ORDER BY month

#Work out the utilisation percentage for each facility by month, sorted by name and month, rounded 
#to 1 decimal place. Opening time is 8am, closing time is 8.30pm. You can treat every month as a 
#full month, regardless of if there were some dates the club was not open.

select name, month, 
	round((100*slots)/
		cast(
			25*(cast((month + interval '1 month') as date)
			- cast (month as date)) as numeric),1) as utilisation
	from  (
		select facs.name as name, date_trunc('month', starttime) as month, sum(slots) as slots
			from cd.bookings bks
			inner join cd.facilities facs
				on bks.facid = facs.facid
			group by facs.facid, month
	) as inn
order by name, month 
